In [1]:
import os
import sys
import random
import nltk
from nltk.corpus import stopwords
from statistics import mean

from nltk.util import ngrams
import nltk.collocations as co_loc


import os
import pandas as pd 
import numpy as np 
import nltk
from nltk import FreqDist, word_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from nltk.corpus import PlaintextCorpusReader, stopwords
from nltk.text import Text 
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import random
import re

In [2]:
SPAMDIR = 'corpus/spam'
HAMDIR = 'corpus/ham'

In [3]:
def AllFiles(MainDirec):
   for dirpath,_,corpus in os.walk(MainDirec):
       for find in corpus:
           yield os.path.abspath(os.path.join(dirpath, find))

def processspamham(MainDirec, Label):    
    BothTexts = [] 
    FileDirectory = AllFiles(MainDirec)
    AllEmails = []
    for find in FileDirectory:
        AllEmails.append(find)
    # process all files
    for find in AllEmails:
        if (find.endswith(".txt")):
                       
            with open(find, 'r', encoding = 'latin-1') as final:
                BothTexts.append(final.read())
    emaildocs = [] 
    # process each email
    for Text in BothTexts:
        Tokens = nltk.word_tokenize(Text)
        emaildocs.append((Tokens, Label))
    return BothTexts, emaildocs

In [4]:
def ExtractTokens(emaildocs):
    Tokens = []
    for doc in emaildocs:
        for w in doc[0]:
            Tokens.append(w)
    return Tokens

In [5]:
TextsSpam, SPAM = processspamham(SPAMDIR, 'spam')
TextsHam, HAM = processspamham(HAMDIR, 'ham')

In [6]:
print("Total Spam Emails:",len(SPAM))
print("Total Ham Emails:",len(HAM))

Total Spam Emails: 1500
Total Ham Emails: 3672


In [8]:
#Combine SPAM and HAM to get Email Total
emaildocs = SPAM + HAM
print("Total Emails:",len(emaildocs)) 

Total Emails: 5172


In [9]:
#Original amount of SPAM tokens without any cleaning
print(len(ExtractTokens(SPAM)))
print(ExtractTokens(SPAM)[:1400])

355375
['Subject', ':', 'dobmeos', 'with', 'hgh', 'my', 'energy', 'level', 'has', 'gone', 'up', '!', 'stukm', 'introducing', 'doctor', '-', 'formulated', 'hgh', 'human', 'growth', 'hormone', '-', 'also', 'called', 'hgh', 'is', 'referred', 'to', 'in', 'medical', 'science', 'as', 'the', 'master', 'hormone', '.', 'it', 'is', 'very', 'plentiful', 'when', 'we', 'are', 'young', ',', 'but', 'near', 'the', 'age', 'of', 'twenty', '-', 'one', 'our', 'bodies', 'begin', 'to', 'produce', 'less', 'of', 'it', '.', 'by', 'the', 'time', 'we', 'are', 'forty', 'nearly', 'everyone', 'is', 'deficient', 'in', 'hgh', ',', 'and', 'at', 'eighty', 'our', 'production', 'has', 'normally', 'diminished', 'at', 'least', '90', '-', '95', '%', '.', 'advantages', 'of', 'hgh', ':', '-', 'increased', 'muscle', 'strength', '-', 'loss', 'in', 'body', 'fat', '-', 'increased', 'bone', 'density', '-', 'lower', 'blood', 'pressure', '-', 'quickens', 'wound', 'healing', '-', 'reduces', 'cellulite', '-', 'improved', 'vision', '-'

In [10]:
#Original amount of HAM tokens without any cleaning
print(len(ExtractTokens(HAM)))
print(ExtractTokens(HAM)[:1400])

830750
['Subject', ':', 'christmas', 'tree', 'farm', 'pictures', 'Subject', ':', 'vastar', 'resources', ',', 'inc', '.', 'gary', ',', 'production', 'from', 'the', 'high', 'island', 'larger', 'block', 'a', '-', '1', '#', '2', 'commenced', 'on', 'saturday', 'at', '2', ':', '00', 'p', '.', 'm', '.', 'at', 'about', '6', ',', '500', 'gross', '.', 'carlos', 'expects', 'between', '9', ',', '500', 'and', '10', ',', '000', 'gross', 'for', 'tomorrow', '.', 'vastar', 'owns', '68', '%', 'of', 'the', 'gross', 'production', '.', 'george', 'x', '3', '-', '6992', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'forwarded', 'by', 'george', 'weissman', '/', 'hou', '/', 'ect', 'on', '12', '/', '13', '/', '99', '10', ':', '16', 'am', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'daren', 'j', 'farmer', '12', '/', '10', '/', '99', '10', ':', '38', 'am', 'to'

In [11]:
ORGSPAMTOKENS = ExtractTokens(SPAM)
print("Uncleaned SPAM Tokens:",len(ORGSPAMTOKENS)) 
ORGHAMTOKENS = ExtractTokens(HAM)
print("Uncleaned HAM Tokens:",len(ORGHAMTOKENS)) 

Uncleaned SPAM Tokens: 355375
Uncleaned HAM Tokens: 830750


In [12]:
ORGSPAMTOKENS[:100]

['Subject',
 ':',
 'dobmeos',
 'with',
 'hgh',
 'my',
 'energy',
 'level',
 'has',
 'gone',
 'up',
 '!',
 'stukm',
 'introducing',
 'doctor',
 '-',
 'formulated',
 'hgh',
 'human',
 'growth',
 'hormone',
 '-',
 'also',
 'called',
 'hgh',
 'is',
 'referred',
 'to',
 'in',
 'medical',
 'science',
 'as',
 'the',
 'master',
 'hormone',
 '.',
 'it',
 'is',
 'very',
 'plentiful',
 'when',
 'we',
 'are',
 'young',
 ',',
 'but',
 'near',
 'the',
 'age',
 'of',
 'twenty',
 '-',
 'one',
 'our',
 'bodies',
 'begin',
 'to',
 'produce',
 'less',
 'of',
 'it',
 '.',
 'by',
 'the',
 'time',
 'we',
 'are',
 'forty',
 'nearly',
 'everyone',
 'is',
 'deficient',
 'in',
 'hgh',
 ',',
 'and',
 'at',
 'eighty',
 'our',
 'production',
 'has',
 'normally',
 'diminished',
 'at',
 'least',
 '90',
 '-',
 '95',
 '%',
 '.',
 'advantages',
 'of',
 'hgh',
 ':',
 '-',
 'increased',
 'muscle',
 'strength',
 '-',
 'loss']

In [13]:
ORGHAMTOKENS[:100]

['Subject',
 ':',
 'christmas',
 'tree',
 'farm',
 'pictures',
 'Subject',
 ':',
 'vastar',
 'resources',
 ',',
 'inc',
 '.',
 'gary',
 ',',
 'production',
 'from',
 'the',
 'high',
 'island',
 'larger',
 'block',
 'a',
 '-',
 '1',
 '#',
 '2',
 'commenced',
 'on',
 'saturday',
 'at',
 '2',
 ':',
 '00',
 'p',
 '.',
 'm',
 '.',
 'at',
 'about',
 '6',
 ',',
 '500',
 'gross',
 '.',
 'carlos',
 'expects',
 'between',
 '9',
 ',',
 '500',
 'and',
 '10',
 ',',
 '000',
 'gross',
 'for',
 'tomorrow',
 '.',
 'vastar',
 'owns',
 '68',
 '%',
 'of',
 'the',
 'gross',
 'production',
 '.',
 'george',
 'x',
 '3',
 '-',
 '6992',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'forwarded',
 'by',
 'george',
 'weissman',
 '/']

In [14]:
#Original SPAM Tokens - Bigrams#
ORGSPAMTOKENSbigram = list(nltk.bigrams(ORGSPAMTOKENS))
print(ORGSPAMTOKENSbigram[:150])

[('Subject', ':'), (':', 'dobmeos'), ('dobmeos', 'with'), ('with', 'hgh'), ('hgh', 'my'), ('my', 'energy'), ('energy', 'level'), ('level', 'has'), ('has', 'gone'), ('gone', 'up'), ('up', '!'), ('!', 'stukm'), ('stukm', 'introducing'), ('introducing', 'doctor'), ('doctor', '-'), ('-', 'formulated'), ('formulated', 'hgh'), ('hgh', 'human'), ('human', 'growth'), ('growth', 'hormone'), ('hormone', '-'), ('-', 'also'), ('also', 'called'), ('called', 'hgh'), ('hgh', 'is'), ('is', 'referred'), ('referred', 'to'), ('to', 'in'), ('in', 'medical'), ('medical', 'science'), ('science', 'as'), ('as', 'the'), ('the', 'master'), ('master', 'hormone'), ('hormone', '.'), ('.', 'it'), ('it', 'is'), ('is', 'very'), ('very', 'plentiful'), ('plentiful', 'when'), ('when', 'we'), ('we', 'are'), ('are', 'young'), ('young', ','), (',', 'but'), ('but', 'near'), ('near', 'the'), ('the', 'age'), ('age', 'of'), ('of', 'twenty'), ('twenty', '-'), ('-', 'one'), ('one', 'our'), ('our', 'bodies'), ('bodies', 'begin'),

In [15]:
#Original HAM Tokens - Bigrams#
ORGHAMTOKENSbigram = list(nltk.bigrams(ORGHAMTOKENS))
print(ORGHAMTOKENSbigram[:150])

[('Subject', ':'), (':', 'christmas'), ('christmas', 'tree'), ('tree', 'farm'), ('farm', 'pictures'), ('pictures', 'Subject'), ('Subject', ':'), (':', 'vastar'), ('vastar', 'resources'), ('resources', ','), (',', 'inc'), ('inc', '.'), ('.', 'gary'), ('gary', ','), (',', 'production'), ('production', 'from'), ('from', 'the'), ('the', 'high'), ('high', 'island'), ('island', 'larger'), ('larger', 'block'), ('block', 'a'), ('a', '-'), ('-', '1'), ('1', '#'), ('#', '2'), ('2', 'commenced'), ('commenced', 'on'), ('on', 'saturday'), ('saturday', 'at'), ('at', '2'), ('2', ':'), (':', '00'), ('00', 'p'), ('p', '.'), ('.', 'm'), ('m', '.'), ('.', 'at'), ('at', 'about'), ('about', '6'), ('6', ','), (',', '500'), ('500', 'gross'), ('gross', '.'), ('.', 'carlos'), ('carlos', 'expects'), ('expects', 'between'), ('between', '9'), ('9', ','), (',', '500'), ('500', 'and'), ('and', '10'), ('10', ','), (',', '000'), ('000', 'gross'), ('gross', 'for'), ('for', 'tomorrow'), ('tomorrow', '.'), ('.', 'vastar

In [16]:
#Original SPAM Tokens - Trigrams#
ORGSPAMTOKENStrigram = list(nltk.trigrams(ORGSPAMTOKENS))
print(ORGSPAMTOKENStrigram[:150])

[('Subject', ':', 'dobmeos'), (':', 'dobmeos', 'with'), ('dobmeos', 'with', 'hgh'), ('with', 'hgh', 'my'), ('hgh', 'my', 'energy'), ('my', 'energy', 'level'), ('energy', 'level', 'has'), ('level', 'has', 'gone'), ('has', 'gone', 'up'), ('gone', 'up', '!'), ('up', '!', 'stukm'), ('!', 'stukm', 'introducing'), ('stukm', 'introducing', 'doctor'), ('introducing', 'doctor', '-'), ('doctor', '-', 'formulated'), ('-', 'formulated', 'hgh'), ('formulated', 'hgh', 'human'), ('hgh', 'human', 'growth'), ('human', 'growth', 'hormone'), ('growth', 'hormone', '-'), ('hormone', '-', 'also'), ('-', 'also', 'called'), ('also', 'called', 'hgh'), ('called', 'hgh', 'is'), ('hgh', 'is', 'referred'), ('is', 'referred', 'to'), ('referred', 'to', 'in'), ('to', 'in', 'medical'), ('in', 'medical', 'science'), ('medical', 'science', 'as'), ('science', 'as', 'the'), ('as', 'the', 'master'), ('the', 'master', 'hormone'), ('master', 'hormone', '.'), ('hormone', '.', 'it'), ('.', 'it', 'is'), ('it', 'is', 'very'), ('

In [17]:
#Original HAM Tokens - Trigrams#
ORGHAMTOKENStrigram = list(nltk.trigrams(ORGHAMTOKENS))
print(ORGHAMTOKENStrigram[:150])

[('Subject', ':', 'christmas'), (':', 'christmas', 'tree'), ('christmas', 'tree', 'farm'), ('tree', 'farm', 'pictures'), ('farm', 'pictures', 'Subject'), ('pictures', 'Subject', ':'), ('Subject', ':', 'vastar'), (':', 'vastar', 'resources'), ('vastar', 'resources', ','), ('resources', ',', 'inc'), (',', 'inc', '.'), ('inc', '.', 'gary'), ('.', 'gary', ','), ('gary', ',', 'production'), (',', 'production', 'from'), ('production', 'from', 'the'), ('from', 'the', 'high'), ('the', 'high', 'island'), ('high', 'island', 'larger'), ('island', 'larger', 'block'), ('larger', 'block', 'a'), ('block', 'a', '-'), ('a', '-', '1'), ('-', '1', '#'), ('1', '#', '2'), ('#', '2', 'commenced'), ('2', 'commenced', 'on'), ('commenced', 'on', 'saturday'), ('on', 'saturday', 'at'), ('saturday', 'at', '2'), ('at', '2', ':'), ('2', ':', '00'), (':', '00', 'p'), ('00', 'p', '.'), ('p', '.', 'm'), ('.', 'm', '.'), ('m', '.', 'at'), ('.', 'at', 'about'), ('at', 'about', '6'), ('about', '6', ','), ('6', ',', '500'

In [18]:
#Create a function to remove punctuation and non-alphabetic characters#
def remove_punc_filter(word):

    mPattern = re.compile('^[^a-z]+$')
    if (mPattern.match(word)):
        return True
    else:
        return False

In [19]:
#Use created function and apply to Original SPAM Tokens. Also convert to all lowercase#
ORGSPAMTOKENSpunc = [w for w in ORGSPAMTOKENS if not remove_punc_filter(w)]
ORGSPAMTOKENSpunc = [w.lower() for w in ORGSPAMTOKENSpunc]
ORGSPAMTOKENSpunc[:50]

['subject',
 'dobmeos',
 'with',
 'hgh',
 'my',
 'energy',
 'level',
 'has',
 'gone',
 'up',
 'stukm',
 'introducing',
 'doctor',
 'formulated',
 'hgh',
 'human',
 'growth',
 'hormone',
 'also',
 'called',
 'hgh',
 'is',
 'referred',
 'to',
 'in',
 'medical',
 'science',
 'as',
 'the',
 'master',
 'hormone',
 'it',
 'is',
 'very',
 'plentiful',
 'when',
 'we',
 'are',
 'young',
 'but',
 'near',
 'the',
 'age',
 'of',
 'twenty',
 'one',
 'our',
 'bodies',
 'begin',
 'to']

In [22]:
#Use created function and apply to Original HAM Tokens. Also convert to all lowercase#
ORGHAMTOKENSpunc = [w for w in ORGHAMTOKENS if not remove_punc_filter(w)]
ORGHAMTOKENSpunc = [w.lower() for w in ORGHAMTOKENSpunc]
ORGHAMTOKENSpunc[:50]

['subject',
 'christmas',
 'tree',
 'farm',
 'pictures',
 'subject',
 'vastar',
 'resources',
 'inc',
 'gary',
 'production',
 'from',
 'the',
 'high',
 'island',
 'larger',
 'block',
 'a',
 'commenced',
 'on',
 'saturday',
 'at',
 'p',
 'm',
 'at',
 'about',
 'gross',
 'carlos',
 'expects',
 'between',
 'and',
 'gross',
 'for',
 'tomorrow',
 'vastar',
 'owns',
 'of',
 'the',
 'gross',
 'production',
 'george',
 'x',
 'forwarded',
 'by',
 'george',
 'weissman',
 'hou',
 'ect',
 'on',
 'am']

In [23]:
ORGSPAMTOKENSpuncFD = FreqDist(ORGSPAMTOKENSpunc)

In [24]:
#Return the 50 most frequent with Stopwords included#
ORGSPAMTOKENStop50 = ORGSPAMTOKENSpuncFD.most_common(50)
#Print out Top 50#
for records in ORGSPAMTOKENStop50:
  print(records)

('the', 7297)
('to', 5173)
('and', 4917)
('of', 4511)
('a', 3794)
('in', 3147)
('you', 2797)
('for', 2526)
('this', 2293)
('is', 2264)
('your', 1952)
('subject', 1658)
('with', 1471)
('that', 1350)
('s', 1324)
('be', 1310)
('or', 1303)
('on', 1268)
('as', 1201)
('are', 1168)
('i', 1150)
('we', 1139)
('it', 1089)
('not', 1078)
('our', 1037)
('com', 993)
('http', 983)
('from', 982)
('have', 888)
('all', 885)
('no', 757)
('at', 744)
('company', 731)
('will', 696)
('by', 671)
('can', 650)
('e', 638)
('an', 608)
('more', 603)
('here', 588)
('www', 587)
('any', 555)
('if', 549)
('information', 520)
('font', 515)
('me', 513)
('t', 509)
('only', 508)
('td', 504)
('has', 493)


In [25]:
ORGHAMTOKENSpuncFD = FreqDist(ORGHAMTOKENSpunc)

In [26]:
#Return the 50 most frequent with Stopwords included#
ORGHAMTOKENStop50 = ORGHAMTOKENSpuncFD.most_common(50)
#Print out Top 50#
for records in ORGHAMTOKENStop50:
  print(records)

('the', 18359)
('to', 15172)
('ect', 13897)
('for', 7982)
('and', 7912)
('hou', 7281)
('enron', 6555)
('subject', 6403)
('on', 6049)
('a', 6026)
('of', 5677)
('you', 5365)
('i', 5241)
('is', 4906)
('this', 4878)
('in', 4570)
('be', 3757)
('will', 3441)
('that', 3419)
('from', 3210)
('have', 3209)
('we', 3202)
('at', 2991)
('gas', 2861)
('deal', 2789)
('com', 2717)
('please', 2715)
('if', 2586)
('with', 2516)
('meter', 2459)
('am', 2405)
('cc', 2359)
('by', 2329)
('pm', 2325)
('hpl', 2318)
('it', 2246)
('are', 2220)
('s', 2111)
('your', 2090)
('me', 2059)
('not', 1996)
('as', 1956)
('daren', 1901)
('thanks', 1813)
('or', 1777)
('re', 1753)
('corp', 1710)
('d', 1658)
('can', 1493)
('any', 1445)


In [27]:
# define stopwords
stopwords = nltk.corpus.stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [28]:
nostopORGSPAMTOKENS = [w for w in ORGSPAMTOKENSpunc if not w in stopwords]

In [29]:
len(nostopORGSPAMTOKENS)

179807

In [30]:
nostopORGSPAMTOKENSFD = FreqDist(nostopORGSPAMTOKENS)

In [31]:
#Return the 50 most frequent with Stopwords removed#
nostopORGSPAMTOKENStop50 = nostopORGSPAMTOKENSFD.most_common(50)
#Print out Top 50#
for records in nostopORGSPAMTOKENStop50:
  print(records)

('subject', 1658)
('com', 993)
('http', 983)
('company', 731)
('e', 638)
('www', 587)
('information', 520)
('font', 515)
('td', 504)
('get', 486)
('please', 485)
('statements', 476)
('email', 475)
('us', 471)
('price', 471)
('new', 434)
('may', 423)
('nbsp', 418)
('one', 392)
('p', 391)
('height', 362)
('time', 361)
('free', 314)
('within', 313)
('pills', 311)
('r', 306)
('size', 306)
('width', 306)
('stock', 299)
('b', 298)
('message', 297)
('money', 295)
('investment', 290)
('c', 283)
('u', 283)
('report', 282)
('inc', 268)
('securities', 263)
('business', 258)
('online', 257)
('click', 256)
('looking', 254)
('best', 254)
('mail', 245)
('contact', 243)
('like', 243)
('computron', 242)
('prices', 239)
('align', 233)
('future', 232)


In [32]:
nostopORGHAMTOKENS = [w for w in ORGHAMTOKENSpunc if not w in stopwords]

In [33]:
len(nostopORGHAMTOKENS)

323901

In [34]:
nostopORGHAMTOKENSFD = FreqDist(nostopORGHAMTOKENS)

In [35]:
#Return the 50 most frequent with Stopwords removed#
nostopORGHAMTOKENStop50 = nostopORGHAMTOKENSFD.most_common(50)
#Print out Top 50#
for records in nostopORGHAMTOKENStop50:
  print(records)

('ect', 13897)
('hou', 7281)
('enron', 6555)
('subject', 6403)
('gas', 2861)
('deal', 2789)
('com', 2717)
('please', 2715)
('meter', 2459)
('cc', 2359)
('pm', 2325)
('hpl', 2318)
('daren', 1901)
('thanks', 1813)
('corp', 1710)
('know', 1438)
('mmbtu', 1349)
('e', 1338)
('j', 1300)
('forwarded', 1296)
('need', 1257)
('farmer', 1137)
('let', 1086)
('attached', 1083)
('xls', 1020)
('see', 1018)
('new', 1003)
('may', 960)
('l', 923)
('contract', 883)
('volume', 877)
('would', 875)
('robert', 875)
('day', 874)
('sitara', 861)
('nom', 831)
('texas', 796)
('get', 790)
('volumes', 790)
('month', 780)
('questions', 760)
('pec', 752)
('deals', 745)
('price', 735)
('ena', 732)
('bob', 706)
('th', 706)
('flow', 673)
('message', 669)
('energy', 664)


In [36]:
#Look for commonality between SPAM and HAM#
SPAMmccount = nostopORGSPAMTOKENSFD.most_common(100)
SPAMmc = set([entry[0] for entry in SPAMmccount])
HAMmccount = nostopORGHAMTOKENSFD.most_common(100)
HAMmc = set([entry[0] for entry in HAMmccount])

SmH = SPAMmc - HAMmc
HmS = HAMmc - SPAMmc
Common = HAMmc.intersection(SPAMmc)

print("Common Words in SPAM and HAM: {}".format(len(SPAMmc)))
print("Common Words in SPAM and HAM: {}".format(len(HmS)))
print("Common Words in SPAM and HAM: {}".format(len(Common)))

Common Words in SPAM and HAM: 100
Common Words in SPAM and HAM: 73
Common Words in SPAM and HAM: 27


In [37]:
Common

{'also',
 'com',
 'company',
 'could',
 'e',
 'gas',
 'get',
 'information',
 'l',
 'like',
 'mail',
 'may',
 'message',
 'need',
 'net',
 'new',
 'one',
 'p',
 'please',
 'price',
 'see',
 'subject',
 'th',
 'time',
 'us',
 'would',
 'x'}

In [38]:
#Examine the tokens and identify those that should be added to the stopwords list#
SPAMstopwords = ["also", "com", "company", "could", "e", "gas", "get","information", "l","like","mail","may","message","need",
                        "net","new","one","p","please","price","see","subject","th","time","us","would","x"]
SPAMtotalstop = stopwords + SPAMstopwords
len(SPAMtotalstop)
nostopSPAMTOKENS = [w for w in ORGSPAMTOKENSpunc if not w in SPAMtotalstop]

In [39]:
#Examine the tokens and identify those that should be added to the stopwords list#
HAMstopwords = ["also", "com", "company", "could", "e", "gas", "get","information", "l","like","mail","may","message","need",
                        "net","new","one","p","please","price","see","subject","th","time","us","would","x"]
HAMtotalstop = stopwords + HAMstopwords
len(HAMtotalstop)
nostopHAMTOKENS = [w for w in ORGHAMTOKENSpunc if not w in HAMtotalstop]

In [40]:
LengthnostopSPAMTOKENS = len(nostopSPAMTOKENS)
LengthnostopHAMTOKENS = len(nostopHAMTOKENS)

In [41]:
LengthnostopSPAMTOKENS

168557

In [42]:
LengthnostopHAMTOKENS

292733

In [43]:
nostopSPAMTOKENSFD = FreqDist(nostopSPAMTOKENS)

In [44]:
#Return the 50 most frequent with Stopwords removed#
nostopSPAMTOKENStop50 = nostopSPAMTOKENSFD.most_common(50)
#Print out Top 50#
for records in nostopSPAMTOKENStop50:
  print(records)

('http', 983)
('www', 587)
('font', 515)
('td', 504)
('statements', 476)
('email', 475)
('nbsp', 418)
('height', 362)
('free', 314)
('within', 313)
('pills', 311)
('r', 306)
('size', 306)
('width', 306)
('stock', 299)
('b', 298)
('money', 295)
('investment', 290)
('c', 283)
('u', 283)
('report', 282)
('inc', 268)
('securities', 263)
('business', 258)
('online', 257)
('click', 256)
('looking', 254)
('best', 254)
('contact', 243)
('computron', 242)
('prices', 239)
('align', 233)
('future', 232)
('tr', 228)
('news', 224)
('products', 222)
('v', 222)
('save', 215)
('go', 213)
('want', 213)
('forward', 212)
('n', 212)
('many', 211)
('use', 211)
('windows', 207)
('microsoft', 207)
('today', 204)
('color', 203)
('million', 201)
('international', 200)


In [45]:
nostopHAMTOKENSFD = FreqDist(nostopHAMTOKENS)

In [46]:
#Return the 50 most frequent with Stopwords removed#
nostopHAMTOKENStop50 = nostopHAMTOKENSFD.most_common(50)
#Print out Top 50#
for records in nostopHAMTOKENStop50:
  print(records)

('ect', 13897)
('hou', 7281)
('enron', 6555)
('deal', 2789)
('meter', 2459)
('cc', 2359)
('pm', 2325)
('hpl', 2318)
('daren', 1901)
('thanks', 1813)
('corp', 1710)
('know', 1438)
('mmbtu', 1349)
('j', 1300)
('forwarded', 1296)
('farmer', 1137)
('let', 1086)
('attached', 1083)
('xls', 1020)
('contract', 883)
('volume', 877)
('robert', 875)
('day', 874)
('sitara', 861)
('nom', 831)
('texas', 796)
('volumes', 790)
('month', 780)
('questions', 760)
('pec', 752)
('deals', 745)
('ena', 732)
('bob', 706)
('flow', 673)
('energy', 664)
('file', 652)
('change', 647)
('production', 639)
('sent', 637)
('call', 619)
('following', 611)
('nomination', 584)
('gary', 561)
('ticket', 560)
('daily', 555)
('mary', 550)
('march', 514)
('april', 512)
('july', 507)
('original', 506)


In [47]:
for records in nostopSPAMTOKENStop50:
    #print(records[0], records[1], records[1]/LengthnostopORGSPAMTOKENS)
    print("Words: ", (records[0]),"\t\tCount: ", records[1], "\t\tPercent of SPAM Tokens: ", records[1]/LengthnostopSPAMTOKENS)

Words:  http 		Count:  983 		Percent of SPAM Tokens:  0.0058318550994619035
Words:  www 		Count:  587 		Percent of SPAM Tokens:  0.0034825014683460193
Words:  font 		Count:  515 		Percent of SPAM Tokens:  0.003055346262688586
Words:  td 		Count:  504 		Percent of SPAM Tokens:  0.002990086439602034
Words:  statements 		Count:  476 		Percent of SPAM Tokens:  0.00282397052629081
Words:  email 		Count:  475 		Percent of SPAM Tokens:  0.002818037815101123
Words:  nbsp 		Count:  418 		Percent of SPAM Tokens:  0.0024798732772889883
Words:  height 		Count:  362 		Percent of SPAM Tokens:  0.00214764145066654
Words:  free 		Count:  314 		Percent of SPAM Tokens:  0.0018628713135615846
Words:  within 		Count:  313 		Percent of SPAM Tokens:  0.0018569386023718978
Words:  pills 		Count:  311 		Percent of SPAM Tokens:  0.0018450731799925248
Words:  r 		Count:  306 		Percent of SPAM Tokens:  0.0018154096240440918
Words:  size 		Count:  306 		Percent of SPAM Tokens:  0.0018154096240440918
Words:  width

In [48]:
for records in nostopHAMTOKENStop50:
    #print(records[0], records[1], records[1]/LengthnostopORGHAMTOKENS)
    print("Words: ", records[0],"\t\tCount: ", records[1], "\t\tPercent of HAM Tokens: ", records[1]/LengthnostopHAMTOKENS)

Words:  ect 		Count:  13897 		Percent of HAM Tokens:  0.04747329477715188
Words:  hou 		Count:  7281 		Percent of HAM Tokens:  0.024872494730693157
Words:  enron 		Count:  6555 		Percent of HAM Tokens:  0.022392419030310896
Words:  deal 		Count:  2789 		Percent of HAM Tokens:  0.009527453344856917
Words:  meter 		Count:  2459 		Percent of HAM Tokens:  0.008400146208319527
Words:  cc 		Count:  2359 		Percent of HAM Tokens:  0.008058537985126377
Words:  pm 		Count:  2325 		Percent of HAM Tokens:  0.007942391189240708
Words:  hpl 		Count:  2318 		Percent of HAM Tokens:  0.007918478613617186
Words:  daren 		Count:  1901 		Percent of HAM Tokens:  0.006493972322901757
Words:  thanks 		Count:  1813 		Percent of HAM Tokens:  0.006193357086491786
Words:  corp 		Count:  1710 		Percent of HAM Tokens:  0.005841500616602843
Words:  know 		Count:  1438 		Percent of HAM Tokens:  0.004912326249517479
Words:  mmbtu 		Count:  1349 		Percent of HAM Tokens:  0.004608294930875576
Words:  j 		Count:  1300 	

In [49]:
#Create Bigram Association Measure#
Bigrammeasures = nltk.collocations.BigramAssocMeasures()

In [50]:
#Create the bigram finder for SPAM#
SPAMbcf = BigramCollocationFinder.from_words(nostopSPAMTOKENS)

SPAMbcf.apply_freq_filter(4)
SPAMbcfscores = SPAMbcf.score_ngrams(Bigrammeasures.raw_freq)
# check the top 50 scores
for score in SPAMbcfscores[:50]:
    print("Words: ", score[0], "\t\tScore: ", score[1])

Words:  ('http', 'www') 		Score:  0.002426478876581809
Words:  ('nbsp', 'nbsp') 		Score:  0.001756082512147226
Words:  ('href', 'http') 		Score:  0.0010382244581951506
Words:  ('looking', 'statements') 		Score:  0.0010204263246260908
Words:  ('pills', 'pills') 		Score:  0.001002628191057031
Words:  ('width', 'height') 		Score:  0.0009729646351085983
Words:  ('src', 'http') 		Score:  0.0009314356567807923
Words:  ('www', 'computron') 		Score:  0.0009017721008323594
Words:  ('forward', 'looking') 		Score:  0.0008424449889354936
Words:  ('http', 'nd') 		Score:  0.0008068487217973741
Words:  ('td', 'td') 		Score:  0.000794983299418001
Words:  ('investment', 'advice') 		Score:  0.0007000599203830158
Words:  ('font', 'size') 		Score:  0.0006822617868139561
Words:  ('td', 'tr') 		Score:  0.0006585309420552098
Words:  ('tr', 'td') 		Score:  0.000557674851830538
Words:  ('align', 'center') 		Score:  0.0005398767182614783
Words:  ('size', 'pt') 		Score:  0.000516145873502732
Words:  ('windows', 

In [51]:
#Create the bigram finder for HAM#
HAMbcf = BigramCollocationFinder.from_words(nostopHAMTOKENS)

HAMbcf.apply_freq_filter(4)
HAMbcfscores = HAMbcf.score_ngrams(Bigrammeasures.raw_freq)
# check the top 50 scores
for score in HAMbcfscores[:50]:
    print("Words: ", score[0], "\t\tScore: ", score[1])

Words:  ('hou', 'ect') 		Score:  0.024684610207936924
Words:  ('ect', 'ect') 		Score:  0.021657961350445627
Words:  ('enron', 'enron') 		Score:  0.004973815729692245
Words:  ('ect', 'cc') 		Score:  0.00475518646684863
Words:  ('corp', 'enron') 		Score:  0.004147123829564826
Words:  ('let', 'know') 		Score:  0.0033409284228289944
Words:  ('daren', 'j') 		Score:  0.003276022860422296
Words:  ('j', 'farmer') 		Score:  0.0027875231012560934
Words:  ('ect', 'pm') 		Score:  0.0019369186255051532
Words:  ('farmer', 'hou') 		Score:  0.0019232542965774272
Words:  ('attached', 'file') 		Score:  0.0017148732804296064
Words:  ('enron', 'cc') 		Score:  0.0016875446225741546
Words:  ('enron', 'hpl') 		Score:  0.00142450629071543
Words:  ('pec', 'pec') 		Score:  0.0012195413567995409
Words:  ('teco', 'tap') 		Score:  0.0011375553832331851
Words:  ('ami', 'chokshi') 		Score:  0.001052153327434898
Words:  ('north', 'america') 		Score:  0.0010316568340433092
Words:  ('tenaska', 'iv') 		Score:  0.0010214

In [52]:
#Create Trigram Association Measure#
Trigrammeasures = nltk.collocations.TrigramAssocMeasures()

In [53]:
#Create the Trigram finder for SPAM#
SPAMtcf = TrigramCollocationFinder.from_words(nostopSPAMTOKENS)

SPAMtcf.apply_freq_filter(4)
SPAMtcfscores = SPAMtcf.score_ngrams(Trigrammeasures.raw_freq)
# check the top 50 scores
for score in SPAMtcfscores[:50]:
    print("Words: ", score[0], "\t\tScore: ", score[1])

Words:  ('nbsp', 'nbsp', 'nbsp') 		Score:  0.001453514241473211
Words:  ('forward', 'looking', 'statements') 		Score:  0.0008246468553664339
Words:  ('pills', 'pills', 'pills') 		Score:  0.0005398767182614783
Words:  ('font', 'size', 'pt') 		Score:  0.000516145873502732
Words:  ('src', 'http', 'nd') 		Score:  0.0004983477399336723
Words:  ('ali', 'duty', 'free') 		Score:  0.0004805496063646126
Words:  ('href', 'http', 'www') 		Score:  0.0004508860504161797
Words:  ('border', 'src', 'http') 		Score:  0.00043308791684711995
Words:  ('http', 'www', 'moopid') 		Score:  0.00043308791684711995
Words:  ('www', 'moopid', 'hotlist') 		Score:  0.00043308791684711995
Words:  ('td', 'tr', 'tr') 		Score:  0.0004212224944677468
Words:  ('width', 'height', 'td') 		Score:  0.0004212224944677468
Words:  ('td', 'td', 'width') 		Score:  0.0004093570720883737
Words:  ('pt', 'font', 'family') 		Score:  0.00039155893851931393
Words:  ('family', 'knle', 'font') 		Score:  0.0003796935161399408
Words:  ('font'

In [54]:
#Create the Trigram finder for HAM#
HAMtcf = TrigramCollocationFinder.from_words(nostopHAMTOKENS)

HAMtcf.apply_freq_filter(4)
HAMtcfscores = HAMtcf.score_ngrams(Trigrammeasures.raw_freq)
# check the top 50 scores
for score in HAMtcfscores[:50]:
    print("Words: ", score[0], "\t\tScore: ", score[1])

Words:  ('hou', 'ect', 'ect') 		Score:  0.021422251676442357
Words:  ('ect', 'ect', 'cc') 		Score:  0.004724441726761247
Words:  ('corp', 'enron', 'enron') 		Score:  0.0031188830777534476
Words:  ('daren', 'j', 'farmer') 		Score:  0.0027875231012560934
Words:  ('farmer', 'hou', 'ect') 		Score:  0.0019232542965774272
Words:  ('j', 'farmer', 'hou') 		Score:  0.0019232542965774272
Words:  ('hou', 'ect', 'pm') 		Score:  0.0015884782378481415
Words:  ('enron', 'enron', 'cc') 		Score:  0.0010419050807391036
Words:  ('enron', 'north', 'america') 		Score:  0.0009667512716366108
Words:  ('north', 'america', 'corp') 		Score:  0.0008711009691425292
Words:  ('cotten', 'hou', 'ect') 		Score:  0.0008506044757509403
Words:  ('robert', 'cotten', 'hou') 		Score:  0.0008506044757509403
Words:  ('clynes', 'corp', 'enron') 		Score:  0.000792531077808105
Words:  ('pat', 'clynes', 'corp') 		Score:  0.000792531077808105
Words:  ('ami', 'chokshi', 'corp') 		Score:  0.0007207933509375437
Words:  ('chokshi', 'c

In [55]:
#Create PMI for SPAM (Bigrams)#
SPAMpmibcf = BigramCollocationFinder.from_words(nostopSPAMTOKENS,window_size = 5)

SPAMpmibcf.apply_freq_filter(3)
SPAMpmiscores = SPAMpmibcf.score_ngrams(Bigrammeasures.pmi)

for item in SPAMpmiscores[:50]:
    print(item)

(('ttfkevh', 'vldeo'), 15.362877016654133)
(('xxca', 'vldeo'), 15.362877016654133)
(('ansi', 'lumens'), 14.94783951737529)
(('brightness', 'ansi'), 14.94783951737529)
(('dlp', 'projectors'), 14.94783951737529)
(('eres', 'charterpipeline'), 14.94783951737529)
(('projectors', 'tdp'), 14.94783951737529)
(('taciturnly', 'ello'), 14.94783951737529)
(('vesicopubic', 'ello'), 14.94783951737529)
(('abdul', 'kareem'), 14.362877016654133)
(('gwmvurvtdu', 'dreamer'), 14.362877016654133)
(('hollerin', 'chu'), 14.362877016654133)
(('psc', 'scanner'), 14.362877016654133)
(('shut', 'kaleste'), 14.362877016654133)
(('siberiantigergrowl', 'hotmail'), 14.362877016654133)
(('cadeau', 'spcial'), 14.362877016654132)
(('shahalam', 'selangord'), 14.362877016654132)
(('incest', 'samples'), 14.140484595317686)
(('alleviates', 'mild'), 14.04094892176677)
(('bottles', 'valarx'), 13.94783951737529)
(('erp', 'erp'), 13.94783951737529)
(('guadalajara', 'mexico'), 13.94783951737529)
(('lagoon', 'backyardkatie'), 13.

In [56]:
#Create PMI for HAM (Bigrams)#
HAMpmibcf = BigramCollocationFinder.from_words(nostopHAMTOKENS,window_size = 5)

HAMpmibcf.apply_freq_filter(3)
HAMpmiscores = HAMpmibcf.score_ngrams(Bigrammeasures.pmi)

for item in HAMpmiscores[:50]:
    print(item)

(('ffot', 'tatum'), 16.159225865392745)
(('cuba', 'libre'), 15.744188366113903)
(('electronics', 'recreation'), 15.744188366113903)
(('hemmings', 'badham'), 15.744188366113903)
(('housewares', 'watches'), 15.744188366113903)
(('ochs', 'bros'), 15.744188366113903)
(('sanger', 'heirs'), 15.744188366113903)
(('appliances', 'electronics'), 15.574263364671593)
(('electronics', 'fitness'), 15.574263364671593)
(('executable', 'arserver'), 15.159225865392749)
(('housewares', 'jewelry'), 15.159225865392749)
(('odum', 'luminant'), 15.159225865392749)
(('samir', 'salama'), 15.159225865392749)
(('stern', 'nyu'), 15.159225865392749)
(('unwind', 'hedges'), 15.159225865392749)
(('capitol', 'switchboard'), 15.159225865392747)
(('fitness', 'recreation'), 15.159225865392747)
(('frens', 'mpg'), 15.159225865392747)
(('groetzinger', 'peusa'), 15.159225865392747)
(('hunting', 'mpe'), 15.159225865392747)
(('imai', 'rika'), 15.159225865392747)
(('ssl', 'jreveffo'), 15.159225865392747)
(('tatum', 'br'), 15.159

In [57]:
#Create PMI for SPAM (Trigrams)#
SPAMpmitcf = TrigramCollocationFinder.from_words(nostopSPAMTOKENS, window_size = 5)

SPAMpmitcf.apply_freq_filter(3)
SPAMpmitscores = SPAMpmitcf.score_ngrams(Trigrammeasures.pmi)
SPAMpmit = SPAMpmitcf.nbest(Trigrammeasures.pmi,50)

for item in SPAMpmitscores[:50]:
    print(item)

(('xxca', 'ttfkevh', 'vldeo'), 32.14079153258711)
(('vesicopubic', 'taciturnly', 'ello'), 31.725754033308263)
(('ttfkevh', 'vldeo', 'vldeo'), 30.55582903186595)
(('xxca', 'vldeo', 'vldeo'), 30.55582903186595)
(('taciturnly', 'ello', 'ello'), 29.725754033308263)
(('vesicopubic', 'ello', 'ello'), 29.725754033308263)
(('shahalam', 'selangord', 'malaysia'), 29.555829031865954)
(('gwmvurvtdu', 'dreamer', 'tw'), 29.14079153258711)
(('baune', 'mercier', 'ebel'), 28.970866531144793)
(('compet', 'itive', 'popularly'), 28.970866531144793)
(('dkny', 'baune', 'ebel'), 28.970866531144793)
(('dkny', 'baune', 'mercier'), 28.970866531144793)
(('dkny', 'mercier', 'ebel'), 28.970866531144793)
(('everyy', 'offf', 'lowerr'), 28.970866531144793)
(('everyy', 'offf', 'wontt'), 28.970866531144793)
(('everyy', 'wontt', 'lowerr'), 28.970866531144793)
(('hoooooooooook', 'discreeeeet', 'nauauauaghty'), 28.970866531144793)
(('kunstmarkt', 'infos', 'thierryehrmann'), 28.970866531144793)
(('lowesst', 'everyy', 'offf

In [58]:
#Create PMI for HAM (Trigrams)#
HAMpmitcf = TrigramCollocationFinder.from_words(nostopHAMTOKENS, window_size = 5)

HAMpmitcf.apply_freq_filter(3)
HAMpmitscores = HAMpmitcf.score_ngrams(Trigrammeasures.pmi)
HAMpmit = HAMpmitcf.nbest(Trigrammeasures.pmi,50)

for item in HAMpmitscores[:50]:
    print(item)

(('ffot', 'tatum', 'tatum'), 32.14852672934319)
(('electronics', 'fitness', 'recreation'), 32.14852672934318)
(('deinetol', 'dukepower', 'mxsdo'), 31.733489230064343)
(('housewares', 'jewelry', 'watches'), 31.733489230064336)
(('appliances', 'electronics', 'fitness'), 31.563564228622027)
(('nomform', 'wk', 'kaf'), 31.14852672934319)
(('tatum', 'tatum', 'br'), 31.14852672934319)
(('executable', 'rename', 'arserver'), 31.148526729343182)
(('alvestad', 'dheineke', 'tsteel'), 30.674595541010767)
(('houstonexp', 'patb', 'cnrl'), 30.563564228622027)
(('ilene', 'erskine', 'azurix'), 30.563564228622027)
(('llipperdt', 'houstonexp', 'cnrl'), 30.563564228622027)
(('llipperdt', 'houstonexp', 'patb'), 30.563564228622027)
(('llipperdt', 'patb', 'cnrl'), 30.563564228622027)
(('magazine', 'kits', 'jigs'), 30.563564228622027)
(('shopnotes', 'magazine', 'kits'), 30.563564228622027)
(('spinexp', 'houstonexp', 'patb'), 30.563564228622027)
(('spinexp', 'llipperdt', 'houstonexp'), 30.563564228622027)
(('sp

In [60]:
random.seed(188)
random.shuffle(emaildocs)
print([doc[1] for doc in emaildocs[:50]])

['ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'spam']


In [65]:
#Print POS Tag frequencies
def getPosStats(emaildocs):
    DOC = emaildocs
    # get list of tags
    POS = [t[1] for t in nltk.pos_tag(DOC)]
    # aggregate class counts
    NounCount, VerbCount, AdjCount, AdvCount = 0, 0, 0, 0
    for tag in POS:
        if tag.startswith('N'): NounCount += 1
        if tag.startswith('V'): VerbCount += 1
        if tag.startswith('J'): AdjCount += 1
        if tag.startswith('R'): AdvCount += 1
    # normalize class counts
    NormNounCount = NounCount / len(POS); print("Noun Frequency: {:.2f}".format(NormNounCount))
    NormVerbCount = VerbCount / len(POS); print("Verb Frequency: {:.2f}".format(NormVerbCount))
    NormAdjCount = AdjCount / len(POS); print("Adjective Frequency: {:.2f}".format(NormAdjCount))
    NormAdvCount = AdvCount / len(POS); print("Adverb Frequency: {:.2f}".format(NormAdvCount))

In [66]:
getPosStats(nostopSPAMTOKENS)

Noun Frequency: 0.58
Verb Frequency: 0.15
Adjective Frequency: 0.20
Adverb Frequency: 0.03


In [67]:
getPosStats(nostopHAMTOKENS)

Noun Frequency: 0.58
Verb Frequency: 0.19
Adjective Frequency: 0.18
Adverb Frequency: 0.03


In [68]:
def gwFEAT(emaildocs, n):
    AWL = []
    for email in emaildocs:
        AWL.extend(email[0])
    AW = nltk.FreqDist(AWL)
    wordITEM = AW.most_common(n)
    wordFEAT = [word for (word, count) in wordITEM]
    return wordFEAT

In [69]:
# Unigram Bag of Words
def gbREP(email, wordFEAT, use_bool=True):
    emailwords = email
    features = {}
    for word in wordFEAT:
        if use_bool:
            features[word] = (word in set(emailwords))
        else:
            features[word] = emailwords.count(word)
    return features

In [70]:
def CVevalMET(num_folds, featuresets):
    accuracyLIST = []
    reference = []
    hypothesis = []
    subsetSIZE = int(len(featuresets) / 10)
    for i in range(num_folds):
        roundTEST = featuresets[(i * subsetSIZE):][:subsetSIZE]
        roundTRAIN = featuresets[:(i * subsetSIZE)] + featuresets[((i + 1) * subsetSIZE):]
        clf = nltk.NaiveBayesClassifier.train(roundTRAIN)
        correct = 0
        for case in roundTEST:
            features = case[0]
            label = case[1]; reference.append(label)
            pred = clf.classify(features); hypothesis.append(pred)
            if label == pred:
                correct += 1
        accuracy = float(correct) / float(len(roundTEST))
        accuracyLIST.append(accuracy)
        print("Accuracy for Fold {}: {}".format(i, accuracy))
    print("---Average Accuracy: {}---".format(mean(accuracyLIST)))
    labels = set(reference)
    recallLIST = []
    precisionLIST = []
    f1LIST = []
    unable_to_calc = False
    for label in labels:
        TP = FP = FN = TN = 0
        for i, val in enumerate(reference):
            if val == label and hypothesis[i] == label: TP += 1
            elif val == label and hypothesis[i] != label: FN += 1
            elif val != label and hypothesis[i] == label: FP += 1
            else: TN += 1
        try:
            recall = TP / (TP + FP)
            precision = TP / (TP + FN)
            recallLIST.append(recall)
            precisionLIST.append(precision)
            f1LIST.append(2 * (recall * precision) / (recall + precision))
        except ZeroDivisionError:
            unable_to_calc = True
    print("\n\tPrecision\tRecall\t\tF1")
    if not unable_to_calc:
        for i, label in enumerate(labels):
            print("{}\t{}\t\t{}\t\t{}".format(label, round(precisionLIST[i], 3),
                                              round(recallLIST[i], 3), round(f1LIST[i], 3)))
    print("\n")
    print(nltk.ConfusionMatrix(reference, hypothesis).pretty_format(sort_by_count=True))
    print(nltk.ConfusionMatrix(reference, hypothesis).pretty_format(sort_by_count=True, show_percents=True))

In [71]:
# Additional Word Featuresets
def getFeatureSets1(emaildocs, stopwords):
    #get word features based on specified stopwords
    word_features = getWordFeatures(emaildocs, stopwords)
    featuresets = [(document_features(d, word_features), c) for (d, c) in emaildocs]
    return featuresets

In [72]:
#Provide a Baseline from emaildocs
wordFEAT = gwFEAT(emaildocs, 2000)
bowFEAT = [(gbREP(email, wordFEAT), label) for (email, label) in emaildocs]
CVevalMET(10, bowFEAT)

Accuracy for Fold 0: 0.9458413926499033
Accuracy for Fold 1: 0.9516441005802708
Accuracy for Fold 2: 0.9439071566731141
Accuracy for Fold 3: 0.9497098646034816
Accuracy for Fold 4: 0.9516441005802708
Accuracy for Fold 5: 0.9516441005802708
Accuracy for Fold 6: 0.9342359767891683
Accuracy for Fold 7: 0.9671179883945842
Accuracy for Fold 8: 0.9458413926499033
Accuracy for Fold 9: 0.941972920696325
---Average Accuracy: 0.9483558994197292---

	Precision	Recall		F1
spam	0.998		0.85		0.918
ham	0.928		0.999		0.962


     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<3406> 264 |
spam |    3<1497>|
-----+-----------+
(row = reference; col = test)

     |             s |
     |      h      p |
     |      a      a |
     |      m      m |
-----+---------------+
 ham | <65.9%>  5.1% |
spam |   0.1% <29.0%>|
-----+---------------+
(row = reference; col = test)



In [73]:
for n in [100, 500, 2000, 5000, 7000, 10000,15000]:
    print("\n=====Results for vocab size {}=====".format(n))
    wordFEAT = gwFEAT(emaildocs, n)
    bowFEAT = [(gbREP(email, wordFEAT), label) for (email, label) in emaildocs]
    CVevalMET(5, bowFEAT)


=====Results for vocab size 100=====
Accuracy for Fold 0: 0.8607350096711799
Accuracy for Fold 1: 0.8994197292069632
Accuracy for Fold 2: 0.8762088974854932
Accuracy for Fold 3: 0.8704061895551257
Accuracy for Fold 4: 0.8800773694390716
---Average Accuracy: 0.8773694390715667---

	Precision	Recall		F1
spam	0.949		0.728		0.824
ham	0.847		0.974		0.906


     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<1528> 277 |
spam |   40 <740>|
-----+-----------+
(row = reference; col = test)

     |             s |
     |      h      p |
     |      a      a |
     |      m      m |
-----+---------------+
 ham | <59.1%> 10.7% |
spam |   1.5% <28.6%>|
-----+---------------+
(row = reference; col = test)


=====Results for vocab size 500=====
Accuracy for Fold 0: 0.9032882011605415
Accuracy for Fold 1: 0.9129593810444874
Accuracy for Fold 2: 0.8994197292069632
Accuracy for Fold 3: 0.9284332688588007
Accuracy for Fold 4: 0.9284332688588007
---Averag

MemoryError: 

In [338]:
BigramMeas = co_loc.BigramAssocMeasures()
emailTOKENS = [token for email in emaildocs for token in email[0]]
emailbigramFINDER = co_loc.BigramCollocationFinder.from_words(emailTOKENS)
emailbigramFREQ = emailbigramFINDER.score_ngrams(BigramMeas.raw_freq)
emailbigramPMI = emailbigramFINDER.score_ngrams(BigramMeas.pmi)

In [337]:
# Bigram Bag of Words
def getbigramREP(email, bigramFEAT, use_bool=True):
    emailbigram = [" ".join(bigram) for bigram in ngrams(email, 2)]
    features = {}
    for bigram in bigramFEAT:
        if use_bool:
            features[bigram] = (bigram in set(emailbigram))
        else:
            features[bigram] = emailbigram.count(bigram)
    return features

In [339]:
for i in [100, 500, 2000, 5000, 7000]:
    print("\n=====Results for {} bigrams=====".format(i))
    emailbigramFREQLIST = [" ".join(bigram[0]) for bigram in emailbigramFREQ[:i]]
    emailbigramPMILIST = [" ".join(bigram[0]) for bigram in emailbigramPMI[:i]]
    
    print("\n-----Frequency-----".format(n))
    bowFEAT = [(getbigramREP(email, emailbigramFREQLIST), label) for (email, label) in emaildocs]
    CVevalMET(5, bowFEAT)
    print("\n-----PMI-----".format(n))
    bowFEAT = [(getbigramREP(email, emailbigramPMILIST), label) for (email, label) in emaildocs]
    CVevalMET(5, bowFEAT)


=====Results for 100 bigrams=====

-----Frequency-----
Accuracy for Fold 0: 0.9690522243713733
Accuracy for Fold 1: 0.9864603481624759
Accuracy for Fold 2: 0.9342359767891683
Accuracy for Fold 3: 0.6112185686653772
Accuracy for Fold 4: 0.690522243713733
Accuracy for Fold 5: 0.746615087040619
Accuracy for Fold 6: 0.7330754352030948
Accuracy for Fold 7: 0.6827852998065764
Accuracy for Fold 8: 0.620889748549323
Accuracy for Fold 9: 0.6808510638297872
---Average Accuracy: 0.7655705996131528---

	Precision	Recall		F1
ham	0.681		0.983		0.805
spam	0.972		0.555		0.706


     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<2500>1170 |
spam |   42<1458>|
-----+-----------+
(row = reference; col = test)

     |             s |
     |      h      p |
     |      a      a |
     |      m      m |
-----+---------------+
 ham | <48.4%> 22.6% |
spam |   0.8% <28.2%>|
-----+---------------+
(row = reference; col = test)


-----PMI-----
Accuracy for Fold

In [378]:
# get feature sets for documents using basic stopwords
Basicstop = getFeatureSets1(emaildocs, stopwords)

In [379]:
# get feature sets for documents using extensive stopwords
Extstop = getFeatureSets1(emaildocs, HAMtotalstop)

In [390]:
# 5 Fold Cross Validation for extensive english stop word list
CVevalMET(5, Basicstop)

Accuracy for Fold 0: 0.8701657458563536
Accuracy for Fold 1: 0.856353591160221
Accuracy for Fold 2: 0.8867403314917127
Accuracy for Fold 3: 1.0
Accuracy for Fold 4: 0.9226519337016574
---Average Accuracy: 0.907182320441989---

	Precision	Recall		F1
ham	0.932		0.663		0.775
spam	0.902		0.985		0.942


     |    s      |
     |    p    h |
     |    a    a |
     |    m    m |
-----+-----------+
spam |<1353> 147 |
 ham |   21 <289>|
-----+-----------+
(row = reference; col = test)

     |      s        |
     |      p      h |
     |      a      a |
     |      m      m |
-----+---------------+
spam | <74.8%>  8.1% |
 ham |   1.2% <16.0%>|
-----+---------------+
(row = reference; col = test)

Wall time: 1min 51s


In [391]:
# 5 Fold Cross Validation for extensive stop word list
CVevalMET(5, Extstop)

Accuracy for Fold 0: 0.856353591160221
Accuracy for Fold 1: 0.856353591160221
Accuracy for Fold 2: 0.8729281767955801
Accuracy for Fold 3: 0.9972375690607734
Accuracy for Fold 4: 0.9226519337016574
---Average Accuracy: 0.9011049723756906---

	Precision	Recall		F1
ham	0.935		0.646		0.764
spam	0.894		0.985		0.937


     |    s      |
     |    p    h |
     |    a    a |
     |    m    m |
-----+-----------+
spam |<1341> 159 |
 ham |   20 <290>|
-----+-----------+
(row = reference; col = test)

     |      s        |
     |      p      h |
     |      a      a |
     |      m      m |
-----+---------------+
spam | <74.1%>  8.8% |
 ham |   1.1% <16.0%>|
-----+---------------+
(row = reference; col = test)

Wall time: 1min 56s
